# IRIS-Flower-classification

Этот проект представляет собой применение машинного обучения с программированием на Python на примере классификации цветов IRIS с использованием машинного обучения с инструментами scikit.
Здесь используются алгоритмы, которые представляют собой некоторые типы машинного обучения с учителем и без учителя.

[Ирисы Фишера](https://ru.wikipedia.org/wiki/%D0%98%D1%80%D0%B8%D1%81%D1%8B_%D0%A4%D0%B8%D1%88%D0%B5%D1%80%D0%B0) — набор данных для задачи классификации, на примере которого Рональд Фишер в 1936 году продемонстрировал работу разработанного им метода дискриминантного анализа. Иногда его также называют ирисами Андерсона, так как данные были собраны американским ботаником Эдгаром Андерсоном. Этот набор данных стал классическим и часто используется в литературе для иллюстрации работы различных статистических алгоритмов.

Ирисы Фишера состоят из данных о 150 экземплярах ириса, по 50 экземпляров из трёх видов — Ирис щетинистый (Iris setosa), Ирис виргинский (Iris virginica) и Ирис разноцветный (Iris versicolor). Для каждого экземпляра измерялись четыре характеристики (в сантиметрах):

- Длина наружной доли околоцветника (англ. sepal length);
- Ширина наружной доли околоцветника (англ. sepal width);
- Длина внутренней доли околоцветника (англ. petal length);
- Ширина внутренней доли околоцветника (англ. petal width).

На основании этого набора данных требуется построить правило классификации, определяющее вид растения по данным измерений. Это задача многоклассовой классификации, так как имеется три класса — три вида ириса.

Один из классов (Iris setosa) линейно-разделим от двух остальных.

В качестве задания вам предлагается повторить данные действия по классификации для наборов данных (можно выбрать, что больше понравится, но лучше сделать для всех)
- [User Knowledge Modeling Data Set](http://archive.ics.uci.edu/ml/datasets/User+Knowledge+Modeling)
- [Blood Transfusion Service Center Data Set](http://archive.ics.uci.edu/ml/datasets/Blood+Transfusion+Service+Center)
- [Somerville Happiness Survey Data Set](http://archive.ics.uci.edu/ml/datasets/Somerville+Happiness+Survey)
- [Wine Quality Data Set](https://archive.ics.uci.edu/ml/datasets/wine+quality)



In [ ]:
# Подключаем библиотеки
import pandas as pd # Работа с набором данных
import numpy as np # Линейная алгебра
# Визуализация данных
import seaborn as sns
import matplotlib.pyplot as plt

# отключение информационных сообщений
from warnings import filterwarnings
filterwarnings(action='ignore')

# разделение выборки
from sklearn.model_selection import train_test_split
#метрики
from sklearn import metrics
#SVC - метод опорных векторов
from sklearn.svm import SVC

In [ ]:
# читаем данные
url = 'https://raw.githubusercontent.com/yakushinav/omo/main/data/iris.csv'
data = pd.read_csv(url)
data.head(5)

In [ ]:
# Разделим выборку на обучающую и тестовую
train, test = train_test_split(data, test_size = 0.4, stratify = data['species'], random_state = 42)

In [ ]:
# Выделим признаки и результат
X_train = train[['sepal_length','sepal_width','petal_length','petal_width']]
y_train = train.species
X_test = test[['sepal_length','sepal_width','petal_length','petal_width']]
y_test = test.species

fn = ['sepal_length','sepal_width','petal_length','petal_width']
cn = data['species'].unique()

## Метод опорных векторов

Рассмотрим подход к построению функции потерь,
основанный на максимизации зазора между классами.
Будем рассматривать линейные классификаторы вида
$$
    a(x) = sign (\langle w, x \rangle + b), \qquad w \in R^d, b \in R.
$$

Будем считать, что существуют такие параметры $w_*$ и $b_*$,
что соответствующий им классификатор $a(x)$ не допускает ни одной ошибки
на обучающей выборке.
В этом случае говорят, что выборка \emph{линейно разделима}.

Пусть задан некоторый классификатор $a(x) = sign (\langle w, x \rangle + b)$.
Заметим, что если одновременно умножить параметры $w$ и $b$
на одну и ту же положительную константу,
то классификатор не изменится.
Распорядимся этой свободой выбора и отнормируем параметры так, что
$$
    \min_{x \in X} | \langle w, x \rangle + b| = 1.
$$
Можно показать, что расстояние от произвольной точки $x_0 \in R^d$ до гиперплоскости,
определяемой данным классификатором, равно
$$
    \rho(x_0, a)
    =
    \frac{
        |\langle w, x \rangle + b|
    }{
        \|w\|
    }.
$$
Тогда расстояние от гиперплоскости до ближайшего объекта обучающей выборки равно
$$
    \min_{x \in X}
    \frac{
        |\langle w, x \rangle + b|
    }{
        \|w\|
    }
    =
    \frac{1}{\|w\|} \min_{x \in X} |\langle w, x \rangle + b|
    =
    \frac{1}{\|w\|}.
$$
Данная величина также называется \emph{отступом (margin)}.

Таким образом, если классификатор без ошибок разделяет обучающую выборку,
то ширина его разделяющей полосы равна $\frac{2}{\|w\|}$.
Известно, что максимизация ширины разделяющей полосы приводит
к повышению обобщающей способности классификатора \cite{mohri12foundations}.
Вспомним также, что на повышение обобщающей способности направлена и регуляризация,
которая штрафует большую норму весов --- а чем больше норма весов,
тем меньше ширина разделяющей полосы.

Итак, требуется построить классификатор, идеально разделяющий обучающую выборку,
и при этом имеющий максимальный отступ.
Запишем соответствующую оптимизационную задачу,
которая и будет определять метод опорных векторов для линейно разделимой выборки (hard margin support vector machine):
$$
    \left\{
        \begin{aligned}
            & \frac{1}{2} \|w\|^2 \to \min_{w, b} \\
            & y_i \left(
                \langle w, x_i \rangle + b
            \right) \geq 1, \quad i = 1, \dots, \ell.
        \end{aligned}
    \right.
$$
Здесь мы воспользовались тем, что линейный классификатор дает правильный ответ
на объекте $x_i$ тогда и только тогда, когда $y_i (\langle w, x_i \rangle + b) \geq 0$.
Из условия нормировки следует,
что $y_i (\langle w, x_i \rangle + b) \geq 1$.

В данной задаче функционал является строго выпуклым, а ограничения линейными,
поэтому сама задача является выпуклой и имеет единственное решение.
Более того, задача является квадратичной и может быть решена крайне эффективно.


In [ ]:
# Метод опорных векторов, линейная модель
linear_svc = SVC(kernel='linear').fit(X_train, y_train)
y_pred=linear_svc.predict(X_test)

In [ ]:
#Выполняем предсказание
y_pred=linear_svc.predict(X_test)

print('accuracy (точность) классификатора',metrics.accuracy_score(y_pred,y_test))

# матрица ошибок
print("матрица ошибок")
print(metrics.confusion_matrix(y_test, y_pred))

In [ ]:
# Использование построенной модели для предсказания класса
new_iris=[1.3, 2.4, 5.6, 3.5]
y_new_iris=linear_svc.predict([new_iris])

print(y_new_iris)

In [ ]:
# Опорные вектора
print(linear_svc.support_vectors_)

### Классификация для набора данных Somerville Happiness Survey Data Set

In [ ]:
url = 'https://raw.githubusercontent.com/setarets/mo2025/refs/heads/main/data/winequality-red.csv'
df = pd.read_csv(url, sep=';')
print("Первые 5 строк DataFrame:")
print(df.head(5))

In [ ]:
df1 = pd.read_csv('https://raw.githubusercontent.com/setarets/mo2025/refs/heads/main/data/winequality-red.csv')
df1 = pd.read_csv(url, sep=';')
df2 = pd.read_csv('https://raw.githubusercontent.com/setarets/mo2025/refs/heads/main/data/winequality-white.csv')
df2 = pd.read_csv(url, sep=';')
combined_df = pd.concat([df1, df2], ignore_index=True)
combined_df.to_csv('combined_file.csv', index=False)
print(combined_df.head(5))

In [ ]:
url_red = 'https://raw.githubusercontent.com/setarets/mo2025/refs/heads/main/data/winequality-red.csv'

url_white = 'https://raw.githubusercontent.com/setarets/mo2025/refs/heads/main/data/winequality-white.csv'

df1 = pd.read_csv(url_red, sep=';')
df2 = pd.read_csv(url_white, sep=';')

df1 = df1.assign(wine='red')
df2 = df2.assign(wine='white')

combined_df = pd.concat([df1, df2], ignore_index=True)

combined_df.to_csv('combined_file.csv', index=False)

print(combined_df.head(5))


In [ ]:
print(combined_df.tail(5))

In [ ]:
train, test = train_test_split(combined_df, test_size = 0.4, stratify = combined_df['quality'], random_state = 42)

In [ ]:
print(combined_df.columns.tolist())

In [ ]:
X_train = train[['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']]
y_train = train.quality
X_test = test[['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']]
y_test = test.quality

fn = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']
cn = combined_df['quality'].unique()

In [ ]:
linear_svc = SVC(kernel='linear').fit(X_train, y_train)
y_pred=linear_svc.predict(X_test)

In [ ]:
y_pred=linear_svc.predict(X_test)

print('accuracy (точность) классификатора',metrics.accuracy_score(y_pred,y_test))

print("матрица ошибок")
print(metrics.confusion_matrix(y_test, y_pred))

In [ ]:
new_quality = [7.4, 0.7, 0, 1.9, 0.076, 11, 34, 0.9978, 3.51, 0.56, 9.4, 12.0]
y_new_quality=linear_svc.predict([new_quality])
print(y_new_quality)